In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import api_key

In [3]:
weather_csv_file = "../WeatherPy/cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Port Alfred,81,ZA,1593293617,71,-33.59,26.89,61.142,12.37
1,1,Butaritari,46,KI,1593293617,75,3.07,172.79,83.102,7.81
2,2,Kuvshinovo,1,RU,1593293617,63,57.03,34.17,62.780,1.49
3,3,Mayo,75,US,1593293617,55,38.89,-76.51,90.266,3.60
4,4,Bredasdorp,68,ZA,1593293524,82,-34.53,20.04,55.670,8.70


In [6]:
# Configure gmaps with API key.
gmaps.configure(api_key=api_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,Rikitea,0,PF,1593293527,75,-23.12,-134.97,70.808,7.67
65,65,Hellín,0,ES,1593293622,36,38.51,-1.70,77.270,2.60
97,97,Khandyga,0,RU,1593293624,56,62.67,135.60,72.968,0.93
98,98,Asău,0,RO,1593293625,60,46.43,26.40,71.870,3.10
135,135,Alcázar de San Juan,0,ES,1593293627,34,39.39,-3.21,79.070,3.10
148,148,Caravelas,0,BR,1593293536,87,-17.71,-39.25,71.690,1.79
180,180,Heishan,0,CN,1593293631,53,41.69,122.11,72.734,1.43
182,182,Teplodar,0,UA,1593293631,53,46.51,30.33,73.670,2.16
183,183,Palpa,0,PE,1593293631,42,-14.53,-75.19,74.822,2.76
204,204,Al Bardīyah,0,LY,1593293632,90,31.76,25.09,71.168,4.06


In [14]:

hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": api_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 9: Rikitea.
Closest hotel is Pension Maro'i.
------------
Retrieving Results for Index 65: Hellín.
Closest hotel is Hotel-restaurante Emilio.
------------
Retrieving Results for Index 97: Khandyga.
Closest hotel is Apel'sin.
------------
Retrieving Results for Index 98: Asău.
Closest hotel is Casa Mika Ciobanus.
------------
Retrieving Results for Index 135: Alcázar de San Juan.
Closest hotel is Hotel Intur Alcázar de San Juan.
------------
Retrieving Results for Index 148: Caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 180: Heishan.
Closest hotel is Heishan Hotel.
------------
Retrieving Results for Index 182: Teplodar.
Missing field/result... skipping.
------------
Retrieving Results for Index 183: Palpa.
Closest hotel is Palpa Lines.
------------
Retrieving Results for Index 204: Al Bardīyah.
Closest hotel is بورتو بردية.
------------
Retrieving Results for Index 216: Kiryat Gat.
Closest hotel is רגע בנחת.
-

In [15]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
9,9,Rikitea,0,PF,1593293527,75,-23.12,-134.97,70.808,7.67,Pension Maro'i
65,65,Hellín,0,ES,1593293622,36,38.51,-1.70,77.270,2.60,Hotel-restaurante Emilio
97,97,Khandyga,0,RU,1593293624,56,62.67,135.60,72.968,0.93,Apel'sin
98,98,Asău,0,RO,1593293625,60,46.43,26.40,71.870,3.10,Casa Mika Ciobanus
135,135,Alcázar de San Juan,0,ES,1593293627,34,39.39,-3.21,79.070,3.10,Hotel Intur Alcázar de San Juan
148,148,Caravelas,0,BR,1593293536,87,-17.71,-39.25,71.690,1.79,Pousada dos Navegantes
180,180,Heishan,0,CN,1593293631,53,41.69,122.11,72.734,1.43,Heishan Hotel
182,182,Teplodar,0,UA,1593293631,53,46.51,30.33,73.670,2.16,NaN
183,183,Palpa,0,PE,1593293631,42,-14.53,-75.19,74.822,2.76,Palpa Lines
204,204,Al Bardīyah,0,LY,1593293632,90,31.76,25.09,71.168,4.06,بورتو بردية


In [17]:
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))